In [1]:
import torch

#imports necessary to define a neural network 
import torch.nn as nn
import torch.nn.functional as F

#ensure you are using GPU.
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

dtype = torch.cuda.FloatTensor

cuda:0


In [2]:
a0=9.0
a1=8.97
a2=0.876
a3=2.9876

def output(x0,x1,x2,x3):
    
    out=a0*x0**(0.87)
    out=out+a1*x1**(0.020)
    out=out+a2*x2**(0.12)
    out=out+a3*x3**(0.987)
    
    return out

In [3]:
xtrain=1000.0*torch.rand(10000,4).type(dtype)
xtest=1000.0*torch.rand(1000,4).type(dtype)

In [4]:
ytest=torch.empty(1000,1).type(dtype)

for i in range(1000):
    
    ytest[i]=output(xtest[i][0],xtest[i][1],xtest[i][2],xtest[i][3]).type(dtype)

ytrain=torch.empty(10000,1).type(dtype)

for i in range(10000):
    
    ytrain[i]=output(xtrain[i][0],xtrain[i][1],xtrain[i][2],xtrain[i][3]).type(dtype)

In [5]:
class Forward(nn.Module):
    def __init__(self):
    
        super(Forward, self).__init__()


        self.fc1 = nn.Linear(4,80)
        self.fc2 = nn.Linear(80,160)
        self.fc4=nn.Linear(160,16)
        self.fc3=nn.Linear(16,1)
        '''
        
        self.fc3 = nn.Linear(6400, 12800)

        self.fc5=nn.Linear(12800, 12800)
        self.fc6=nn.Linear(12800, 1280)
        self.fc4 = nn.Linear(1280, 784)


        self.dropout = nn.Dropout(0.3)
        '''
    def forward(self, x):
        
        x = F.relu(self.fc1(x)) # (input, negative_slope=0.2)
        
        x = F.relu(self.fc2(x))
        
        x = F.relu(self.fc4(x))
        
        x = F.relu(self.fc3(x))
        
        '''
        out =F.relu(self.fc5(x))
        out =F.relu(self.fc6(out))
        out =F.relu(self.fc4(out))
        out[out>255]=out[out>255]/100
        out[out<55]=out[out<55]*1000
        '''
        return x

In [6]:
import torch.optim as optim

learning_rate=0.001
momentum=0.05

forwardnetwork = Forward().cuda()
forwardoptimizer = optim.Adagrad(forwardnetwork.parameters(), lr=learning_rate,
                      )
print(forwardnetwork)

Forward(
  (fc1): Linear(in_features=4, out_features=80, bias=True)
  (fc2): Linear(in_features=80, out_features=160, bias=True)
  (fc4): Linear(in_features=160, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
)


In [7]:
forwardnetwork(xtrain[0:10].reshape(10,4))

tensor([[11.3573],
        [ 6.7759],
        [15.2011],
        [ 6.4226],
        [10.1378],
        [17.0915],
        [ 6.9825],
        [12.3328],
        [16.1071],
        [ 9.8781]], device='cuda:0', grad_fn=<ReluBackward0>)

In [9]:
#training loop

batchsize=8

numbatches=int(len(xtrain)/batchsize)

nepochs=1000
criterion=nn.MSELoss()

for i in range(nepochs):
    
    epochloss=0
    
    for j in range(numbatches):
        
        out=forwardnetwork(xtrain[j*batchsize:(j+1)*batchsize].reshape(batchsize,4)).type(dtype)
        out=out.reshape(batchsize,1)
        #print(out)
        loss=criterion(out,ytrain[j*batchsize:(j+1)*batchsize].reshape(batchsize,1)).type(dtype)
        
        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(forwardnetwork.parameters(), 1.0)
        forwardoptimizer.step()
        
        epochloss=epochloss+loss
        
        testout=forwardnetwork(xtest.reshape(1000,4))
        lossc=nn.MSELoss()
        testloss=lossc(testout,ytest)
        
    print(epochloss,testloss,i)

tensor(4818099.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(3678.8567, device='cuda:0', grad_fn=<MseLossBackward>) 0
tensor(4737202.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(3617.2092, device='cuda:0', grad_fn=<MseLossBackward>) 1
tensor(4669662.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(3563.2542, device='cuda:0', grad_fn=<MseLossBackward>) 2
tensor(4611331., device='cuda:0', grad_fn=<AddBackward0>) tensor(3516.1641, device='cuda:0', grad_fn=<MseLossBackward>) 3
tensor(4560611., device='cuda:0', grad_fn=<AddBackward0>) tensor(3475.1116, device='cuda:0', grad_fn=<MseLossBackward>) 4
tensor(4516154., device='cuda:0', grad_fn=<AddBackward0>) tensor(3439.4504, device='cuda:0', grad_fn=<MseLossBackward>) 5
tensor(4476674., device='cuda:0', grad_fn=<AddBackward0>) tensor(3407.5134, device='cuda:0', grad_fn=<MseLossBackward>) 6
tensor(4441056.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(3377.6467, device='cuda:0', grad_fn=<MseLossBackward>) 7
tensor(4

KeyboardInterrupt: 

In [36]:
xtes=torch.rand(10,4).type(dtype)
ypred=forwardnetwork(xtes)
yreal=output(xtes[0][0],xtes[0][1],xtes[0][2],xtes[0][3])

print(xtes)
print(ypred)
print(yreal)
print(output(xtes[1][0],xtes[1][1],xtes[1][2],xtes[1][3]))

print(output(xtes[2][0],xtes[2][1],xtes[2][2],xtes[2][3]))

print(output(xtes[3][0],xtes[3][1],xtes[3][2],xtes[3][3]))

tensor([[0.8150, 0.0441, 0.4272, 0.7309],
        [0.7415, 0.7666, 0.7173, 0.2409],
        [0.3387, 0.2991, 0.0948, 0.4867],
        [0.0115, 0.1042, 0.9463, 0.6822],
        [0.7188, 0.4683, 0.5739, 0.2014],
        [0.0999, 0.0943, 0.1497, 0.1110],
        [0.1485, 0.2601, 0.2366, 0.6852],
        [0.4403, 0.9630, 0.5905, 0.2375],
        [0.9963, 0.8133, 0.4884, 0.3607],
        [0.6376, 0.3741, 0.0467, 0.2939]], device='cuda:0')
tensor([[11.7099],
        [10.5458],
        [ 9.2100],
        [ 8.7002],
        [10.1352],
        [ 7.2165],
        [ 9.0531],
        [ 9.4150],
        [11.7207],
        [ 9.8663]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor(18.9432, device='cuda:0')
tensor(17.4356, device='cuda:0')
tensor(14.3926, device='cuda:0')
tensor(11.6771, device='cuda:0')


In [53]:
class Generator(nn.Module):
    def __init__(self):
    
        super(Generator, self).__init__()


        self.fc1 = nn.Linear(40,80)
        self.fc2 = nn.Linear(80,16)
        self.fc4=nn.Linear(16,16)
        self.fc3=nn.Linear(16,4)
        '''
        
        self.fc3 = nn.Linear(6400, 12800)

        self.fc5=nn.Linear(12800, 12800)
        self.fc6=nn.Linear(12800, 1280)
        self.fc4 = nn.Linear(1280, 784)


        self.dropout = nn.Dropout(0.3)
        '''
    def forward(self, x):
        
        x = F.relu(self.fc1(x)) # (input, negative_slope=0.2)
        
        x = F.relu(self.fc2(x))
        
        x = F.relu(self.fc4(x))
        
        x = F.relu(self.fc3(x))
        
        '''
        out =F.relu(self.fc5(x))
        out =F.relu(self.fc6(out))
        out =F.relu(self.fc4(out))
        out[out>255]=out[out>255]/100
        out[out<55]=out[out<55]*1000
        '''
        return x

In [54]:
gennetwork = Generator().cuda()
genoptimizer = optim.Adagrad(gennetwork.parameters(), lr=learning_rate,
                            )
print(gennetwork)

Generator(
  (fc1): Linear(in_features=40, out_features=80, bias=True)
  (fc2): Linear(in_features=80, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=4, bias=True)
)


In [55]:
def customloss(inp,bsize):
    
    cusout=forwardnetwork(inp.reshape([bsize,4])).type(dtype)
    
    cusout=cusout.reshape([bsize,1]).type(dtype)
    cusideal=1900*torch.ones([bsize,1]).type(dtype)
    
    cuscriterion= nn.MSELoss()
    
    loss= cuscriterion(cusout,cusideal).type(dtype)
    
    return loss

In [56]:
xgarb=torch.rand([100,40]).type(dtype)

In [73]:
batchsize=8

batches=len(xgarb)/batchsize

epochs=2000

for i in range(epochs):
    
    epochloss=0
        
    for j in range(int(batches)):
        
        
        #print(Ytensor)
        out=gennetwork(xgarb[j*batchsize:(j+1)*batchsize].reshape([batchsize,40])).type(dtype)

        #compute loss
        loss = customloss(out,batchsize).type(dtype)


        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(gennetwork.parameters(), 1.0)
        genoptimizer.step()
        
        epochloss=epochloss+loss
        
    print(epochloss,i)

tensor(220656.7656, device='cuda:0', grad_fn=<AddBackward0>) 0
tensor(220655.7812, device='cuda:0', grad_fn=<AddBackward0>) 1
tensor(220654.7344, device='cuda:0', grad_fn=<AddBackward0>) 2
tensor(220654.4375, device='cuda:0', grad_fn=<AddBackward0>) 3
tensor(220654.1094, device='cuda:0', grad_fn=<AddBackward0>) 4
tensor(220653.1562, device='cuda:0', grad_fn=<AddBackward0>) 5
tensor(220652.2656, device='cuda:0', grad_fn=<AddBackward0>) 6
tensor(220651.9375, device='cuda:0', grad_fn=<AddBackward0>) 7
tensor(220651.0938, device='cuda:0', grad_fn=<AddBackward0>) 8
tensor(220650.8281, device='cuda:0', grad_fn=<AddBackward0>) 9
tensor(220649.2188, device='cuda:0', grad_fn=<AddBackward0>) 10
tensor(220648.6250, device='cuda:0', grad_fn=<AddBackward0>) 11
tensor(220648.3750, device='cuda:0', grad_fn=<AddBackward0>) 12
tensor(220647.6250, device='cuda:0', grad_fn=<AddBackward0>) 13
tensor(220647.4375, device='cuda:0', grad_fn=<AddBackward0>) 14
tensor(220646.5000, device='cuda:0', grad_fn=<AddB

tensor(220568.8594, device='cuda:0', grad_fn=<AddBackward0>) 133
tensor(220568.5312, device='cuda:0', grad_fn=<AddBackward0>) 134
tensor(220567.5938, device='cuda:0', grad_fn=<AddBackward0>) 135
tensor(220566.9531, device='cuda:0', grad_fn=<AddBackward0>) 136
tensor(220566.0469, device='cuda:0', grad_fn=<AddBackward0>) 137
tensor(220565.7031, device='cuda:0', grad_fn=<AddBackward0>) 138
tensor(220564.8281, device='cuda:0', grad_fn=<AddBackward0>) 139
tensor(220563.8750, device='cuda:0', grad_fn=<AddBackward0>) 140
tensor(220563.2656, device='cuda:0', grad_fn=<AddBackward0>) 141
tensor(220562.8594, device='cuda:0', grad_fn=<AddBackward0>) 142
tensor(220562.5625, device='cuda:0', grad_fn=<AddBackward0>) 143
tensor(220561.4688, device='cuda:0', grad_fn=<AddBackward0>) 144
tensor(220560.4688, device='cuda:0', grad_fn=<AddBackward0>) 145
tensor(220560.2031, device='cuda:0', grad_fn=<AddBackward0>) 146
tensor(220559.4219, device='cuda:0', grad_fn=<AddBackward0>) 147
tensor(220558.9688, devic

tensor(220485.2344, device='cuda:0', grad_fn=<AddBackward0>) 260
tensor(220484.5000, device='cuda:0', grad_fn=<AddBackward0>) 261
tensor(220484.1094, device='cuda:0', grad_fn=<AddBackward0>) 262
tensor(220483.7344, device='cuda:0', grad_fn=<AddBackward0>) 263
tensor(220482.5156, device='cuda:0', grad_fn=<AddBackward0>) 264
tensor(220482.2188, device='cuda:0', grad_fn=<AddBackward0>) 265
tensor(220481.7812, device='cuda:0', grad_fn=<AddBackward0>) 266
tensor(220480.6719, device='cuda:0', grad_fn=<AddBackward0>) 267
tensor(220480.3438, device='cuda:0', grad_fn=<AddBackward0>) 268
tensor(220479.4375, device='cuda:0', grad_fn=<AddBackward0>) 269
tensor(220478.6719, device='cuda:0', grad_fn=<AddBackward0>) 270
tensor(220478.2188, device='cuda:0', grad_fn=<AddBackward0>) 271
tensor(220477.9062, device='cuda:0', grad_fn=<AddBackward0>) 272
tensor(220477.1094, device='cuda:0', grad_fn=<AddBackward0>) 273
tensor(220475.8281, device='cuda:0', grad_fn=<AddBackward0>) 274
tensor(220475.5625, devic

tensor(220398.5625, device='cuda:0', grad_fn=<AddBackward0>) 388
tensor(220398.2500, device='cuda:0', grad_fn=<AddBackward0>) 389
tensor(220397.7188, device='cuda:0', grad_fn=<AddBackward0>) 390
tensor(220396.9375, device='cuda:0', grad_fn=<AddBackward0>) 391
tensor(220396.0156, device='cuda:0', grad_fn=<AddBackward0>) 392
tensor(220395.4531, device='cuda:0', grad_fn=<AddBackward0>) 393
tensor(220394.4062, device='cuda:0', grad_fn=<AddBackward0>) 394
tensor(220393.9688, device='cuda:0', grad_fn=<AddBackward0>) 395
tensor(220393.5312, device='cuda:0', grad_fn=<AddBackward0>) 396
tensor(220392.2188, device='cuda:0', grad_fn=<AddBackward0>) 397
tensor(220391.7656, device='cuda:0', grad_fn=<AddBackward0>) 398
tensor(220391.4688, device='cuda:0', grad_fn=<AddBackward0>) 399
tensor(220390.3125, device='cuda:0', grad_fn=<AddBackward0>) 400
tensor(220389.8438, device='cuda:0', grad_fn=<AddBackward0>) 401
tensor(220388.8594, device='cuda:0', grad_fn=<AddBackward0>) 402
tensor(220388.0469, devic

tensor(220306.0938, device='cuda:0', grad_fn=<AddBackward0>) 515
tensor(220304.9844, device='cuda:0', grad_fn=<AddBackward0>) 516
tensor(220304.4375, device='cuda:0', grad_fn=<AddBackward0>) 517
tensor(220303.8281, device='cuda:0', grad_fn=<AddBackward0>) 518
tensor(220302.6406, device='cuda:0', grad_fn=<AddBackward0>) 519
tensor(220302., device='cuda:0', grad_fn=<AddBackward0>) 520
tensor(220301.4062, device='cuda:0', grad_fn=<AddBackward0>) 521
tensor(220300.4219, device='cuda:0', grad_fn=<AddBackward0>) 522
tensor(220299.8125, device='cuda:0', grad_fn=<AddBackward0>) 523
tensor(220298.7188, device='cuda:0', grad_fn=<AddBackward0>) 524
tensor(220297.7969, device='cuda:0', grad_fn=<AddBackward0>) 525
tensor(220297.2969, device='cuda:0', grad_fn=<AddBackward0>) 526
tensor(220296.5781, device='cuda:0', grad_fn=<AddBackward0>) 527
tensor(220296.0938, device='cuda:0', grad_fn=<AddBackward0>) 528
tensor(220294.5312, device='cuda:0', grad_fn=<AddBackward0>) 529
tensor(220294.1094, device='c

...

tensor(219359.5781, device='cuda:0', grad_fn=<AddBackward0>) 1418
tensor(219358.4375, device='cuda:0', grad_fn=<AddBackward0>) 1419
tensor(219357.0938, device='cuda:0', grad_fn=<AddBackward0>) 1420
tensor(219355.4688, device='cuda:0', grad_fn=<AddBackward0>) 1421
tensor(219354.0938, device='cuda:0', grad_fn=<AddBackward0>) 1422
tensor(219353.1875, device='cuda:0', grad_fn=<AddBackward0>) 1423
tensor(219351.5312, device='cuda:0', grad_fn=<AddBackward0>) 1424
tensor(219350.4375, device='cuda:0', grad_fn=<AddBackward0>) 1425
tensor(219349.1719, device='cuda:0', grad_fn=<AddBackward0>) 1426
tensor(219347.5625, device='cuda:0', grad_fn=<AddBackward0>) 1427
tensor(219346.2500, device='cuda:0', grad_fn=<AddBackward0>) 1428
tensor(219345.0938, device='cuda:0', grad_fn=<AddBackward0>) 1429
tensor(219343.4062, device='cuda:0', grad_fn=<AddBackward0>) 1430
tensor(219342.4531, device='cuda:0', grad_fn=<AddBackward0>) 1431
tensor(219341.2812, device='cuda:0', grad_fn=<AddBackward0>) 1432
tensor(219

tensor(219188.3281, device='cuda:0', grad_fn=<AddBackward0>) 1544
tensor(219186.6250, device='cuda:0', grad_fn=<AddBackward0>) 1545
tensor(219185.2031, device='cuda:0', grad_fn=<AddBackward0>) 1546
tensor(219183.8750, device='cuda:0', grad_fn=<AddBackward0>) 1547
tensor(219182.2344, device='cuda:0', grad_fn=<AddBackward0>) 1548
tensor(219181.0156, device='cuda:0', grad_fn=<AddBackward0>) 1549
tensor(219179.5625, device='cuda:0', grad_fn=<AddBackward0>) 1550
tensor(219178.2656, device='cuda:0', grad_fn=<AddBackward0>) 1551
tensor(219176.8438, device='cuda:0', grad_fn=<AddBackward0>) 1552
tensor(219175.2500, device='cuda:0', grad_fn=<AddBackward0>) 1553
tensor(219173.7969, device='cuda:0', grad_fn=<AddBackward0>) 1554
tensor(219172.7344, device='cuda:0', grad_fn=<AddBackward0>) 1555
tensor(219171.1562, device='cuda:0', grad_fn=<AddBackward0>) 1556
tensor(219170.0312, device='cuda:0', grad_fn=<AddBackward0>) 1557
tensor(219168.5000, device='cuda:0', grad_fn=<AddBackward0>) 1558
tensor(219

tensor(219003.4375, device='cuda:0', grad_fn=<AddBackward0>) 1673
tensor(219001.7812, device='cuda:0', grad_fn=<AddBackward0>) 1674
tensor(219000.4219, device='cuda:0', grad_fn=<AddBackward0>) 1675
tensor(218998.6094, device='cuda:0', grad_fn=<AddBackward0>) 1676
tensor(218997.4219, device='cuda:0', grad_fn=<AddBackward0>) 1677
tensor(218995.8906, device='cuda:0', grad_fn=<AddBackward0>) 1678
tensor(218994.6250, device='cuda:0', grad_fn=<AddBackward0>) 1679
tensor(218993.1875, device='cuda:0', grad_fn=<AddBackward0>) 1680
tensor(218991.5469, device='cuda:0', grad_fn=<AddBackward0>) 1681
tensor(218990.4062, device='cuda:0', grad_fn=<AddBackward0>) 1682
tensor(218988.7656, device='cuda:0', grad_fn=<AddBackward0>) 1683
tensor(218987.0469, device='cuda:0', grad_fn=<AddBackward0>) 1684
tensor(218985.4844, device='cuda:0', grad_fn=<AddBackward0>) 1685
tensor(218984.4375, device='cuda:0', grad_fn=<AddBackward0>) 1686
tensor(218982.8594, device='cuda:0', grad_fn=<AddBackward0>) 1687
tensor(218

tensor(218812.6250, device='cuda:0', grad_fn=<AddBackward0>) 1800
tensor(218811.1406, device='cuda:0', grad_fn=<AddBackward0>) 1801
tensor(218809.7500, device='cuda:0', grad_fn=<AddBackward0>) 1802
tensor(218808.0781, device='cuda:0', grad_fn=<AddBackward0>) 1803
tensor(218806.6406, device='cuda:0', grad_fn=<AddBackward0>) 1804
tensor(218804.7812, device='cuda:0', grad_fn=<AddBackward0>) 1805
tensor(218803.2812, device='cuda:0', grad_fn=<AddBackward0>) 1806
tensor(218801.6562, device='cuda:0', grad_fn=<AddBackward0>) 1807
tensor(218800.3438, device='cuda:0', grad_fn=<AddBackward0>) 1808
tensor(218798.7031, device='cuda:0', grad_fn=<AddBackward0>) 1809
tensor(218797.3438, device='cuda:0', grad_fn=<AddBackward0>) 1810
tensor(218795.7188, device='cuda:0', grad_fn=<AddBackward0>) 1811
tensor(218794.3281, device='cuda:0', grad_fn=<AddBackward0>) 1812
tensor(218792.4375, device='cuda:0', grad_fn=<AddBackward0>) 1813
tensor(218791.0625, device='cuda:0', grad_fn=<AddBackward0>) 1814
tensor(218

tensor(218608.5469, device='cuda:0', grad_fn=<AddBackward0>) 1930
tensor(218606.7969, device='cuda:0', grad_fn=<AddBackward0>) 1931
tensor(218605.1875, device='cuda:0', grad_fn=<AddBackward0>) 1932
tensor(218603.7188, device='cuda:0', grad_fn=<AddBackward0>) 1933
tensor(218601.9375, device='cuda:0', grad_fn=<AddBackward0>) 1934
tensor(218600.4531, device='cuda:0', grad_fn=<AddBackward0>) 1935
tensor(218598.7969, device='cuda:0', grad_fn=<AddBackward0>) 1936
tensor(218597.2812, device='cuda:0', grad_fn=<AddBackward0>) 1937
tensor(218595.6719, device='cuda:0', grad_fn=<AddBackward0>) 1938
tensor(218594.1094, device='cuda:0', grad_fn=<AddBackward0>) 1939
tensor(218592.4375, device='cuda:0', grad_fn=<AddBackward0>) 1940
tensor(218590.7969, device='cuda:0', grad_fn=<AddBackward0>) 1941
tensor(218589.3438, device='cuda:0', grad_fn=<AddBackward0>) 1942
tensor(218587.6875, device='cuda:0', grad_fn=<AddBackward0>) 1943
tensor(218586.0312, device='cuda:0', grad_fn=<AddBackward0>) 1944
tensor(218

In [72]:
print(out)
print(output(out[1][0],out[1][1],out[1][2],out[1][3]))

tensor([[224.6277,   0.0000, 283.2135, 328.2939],
        [238.2836,   0.0000, 300.3886, 348.1620],
        [235.2614,   0.0000, 296.6285, 343.8041],
        [225.2778,   0.0000, 284.0018, 329.2142],
        [240.0195,   0.0000, 302.6304, 350.6966],
        [232.7774,   0.0000, 293.4954, 340.2213],
        [233.1950,   0.0000, 294.0080, 340.7433],
        [241.8545,   0.0000, 304.9173, 353.3769]], device='cuda:0',
       grad_fn=<ReluBackward0>)
tensor(2018.4243, device='cuda:0', grad_fn=<AddBackward0>)


In [70]:
print(output(out[0][0],out[0][1],out[0][2],out[0][3]))

tensor(1911.4099, device='cuda:0', grad_fn=<AddBackward0>)


In [ ]:

xtens0=torch.linspace(0,10,1000).type(dtype)
xtens1=torch.linspace(2,54,1000).type(dtype)
xtens2=torch.linspace(1,100,1000).type(dtype)
xtens3=torch.linspace(10,100,1000).type(dtype)

ytens=torch.empty(1000,1).type(dtype)

for i in range(1000):
    
    ytens[i]=out(xtens0[i],xtens1[i],xtens2[i],xtens3[i]).type(dtype)

print(ytens)